In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [127]:
##### 파일 불러오기 #####
df = pd.read_csv("C:\\Users\\asus\\BigContest\\files\\data_preprocessing\\1_food_waste_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'waste_cnt', 'waste_amt']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,waste_cnt,waste_amt
base_date,,,,
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850


In [128]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'waste_amt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,waste_cnt
base_date,,
2019-11-30,50110250,620
2019-12-31,50110250,8178
2020-01-31,50110250,13042
2020-02-29,50110250,12264
2020-03-31,50110250,14316
...,...,...
2021-02-28,50130620,4563
2021-03-31,50130620,4920
2021-04-30,50130620,5256


In [129]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110590  ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,waste_cnt
base_date,
2018-01-31,18416
2018-02-28,17491
2018-03-31,19396
2018-04-30,19179
2018-05-31,21277
2018-06-30,23311
2018-07-31,28626
2018-08-31,26842
2018-09-30,25510


In [130]:
# series 형으로 변환
sub_df.to_dict('series')

{'waste_cnt': base_date
 2018-01-31    18416
 2018-02-28    17491
 2018-03-31    19396
 2018-04-30    19179
 2018-05-31    21277
 2018-06-30    23311
 2018-07-31    28626
 2018-08-31    26842
 2018-09-30    25510
 2018-10-31    23042
 2018-11-30    20791
 2018-12-31    20161
 2019-01-31    19741
 2019-02-28    18910
 2019-03-31    18966
 2019-04-30    18991
 2019-05-31    21797
 2019-06-30    23387
 2019-07-31    26082
 2019-08-31    26561
 2019-09-30    24806
 2019-10-31    22040
 2019-11-30    20387
 2019-12-31    19300
 2020-01-31    20359
 2020-02-29    18554
 2020-03-31    19812
 2020-04-30    18703
 2020-05-31    21611
 2020-06-30    20591
 2020-07-31    25261
 2020-08-31    25377
 2020-09-30    21173
 2020-10-31    21072
 2020-11-30    18837
 2020-12-31    18178
 2021-01-31    17467
 2021-02-28    13784
 2021-03-31    16998
 2021-04-30    17780
 2021-05-31    20131
 2021-06-30    20296
 Name: waste_cnt, dtype: int64}

# ARIMA

In [131]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [132]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : 0.998298
p-value : 0.994248
Critical Values :
	1%: -3.661429
	5%: -2.960525
	10%: -2.619319


In [133]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 또 p-value 확인 -> 0.05보다 크면 이 파일 안되고 log파일로 넘어가세요 / 작으면 아래로 쭉 진행 #####

ADF Statistic : -4.568573
p-value : 0.000148
Critical Values :
	1%: -3.669920
	5%: -2.964071
	10%: -2.621171


In [134]:
##### 이건 설치하는 거라 처음 돌릴 때 한 번만 주석 풀어서 설치하고 그 이후로는 필요없음! #####

#!pip install pmdarima

In [135]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=798.066, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=776.978, Time=0.04 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=762.136, Time=0.09 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=760.855, Time=0.03 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=761.280, Time=0.19 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=763.410, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=764.330, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=760.063, Time=0.03 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=760.952, Time=0.04 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.18 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=763.007, Time=0.02 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=769.104, Time=0.04 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=761.287, Time=0.06 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=759.252, Time=0.08 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept

ARIMA(order=(3, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [136]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.waste_cnt.values, order=(3,0,0))     ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(3, 0)   Log Likelihood                -372.759
Method:                       css-mle   S.D. of innovations           1697.133
Date:                Fri, 10 Sep 2021   AIC                            755.517
Time:                        15:19:55   BIC                            764.206
Sample:                             0   HQIC                           758.702
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.106e+04    619.313     33.998      0.000    1.98e+04    2.23e+04
ar.L1.y        0.8266      0.132      6.240      0.000       0.567       1.086
ar.L2.y        0.1981      0.185      1.073      0.2

# 예측하기

In [137]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([21725.63873209, 21876.93409649]), array([1697.132572  , 2201.89907302]), array([[18399.32001398, 25051.9574502 ],
       [17561.29121578, 26192.5769772 ]]))
